In [24]:
import numpy as np
import itertools
import ipcoal
import shadie
from shadie.postsim.src.ts_utils import stats
from shadie.postsim.src.ts_utils import draw_stats
import tskit
import toytree
from toytree.learn import wright_fisher_simulation
from toytree.utils import toytree_sequence
toytree.set_log_level("ERROR")

# Exploring Forward-Time Simulations

<div class="alert alert-block alert-success">
    <h2>Revisit: Wright Fisher Model</h2>
    <p>
        Recall the Wright Fisher process, in which two random gene copies from each generation are chosen to produce individuals in the next generation:
    </p>
</div>

In [25]:
wright_fisher_simulation(time=11, popsize=10, show_diploids=True, sort_edges=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6c75d8422a1542aca29fc2da592b56c7"> Gene copies at the present 0 5 10 Time (generations)

<div class="alert alert-block alert-success">
    <p>
        When we 'untangle' the lines (which represent a gene copy passing from one generation to another), we can start to see the relationships between all the gene copies in each generation:
    </p>
</div>

In [26]:
wf = wright_fisher_simulation(time=10, popsize=11, sort_edges=True)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1dd1c14d582e4967bf863dc8a3f529b8"> Gene copies at the present 0 3 6 9 Time (generations)

<div class="alert alert-block alert-success">
    <p>
        And when we start with multiple alleles, we can start to see the process of extinction or fixation of those alleles by genetic drift. Remember that there is no selection in the standard Wright Fisher model, so these alleles are going to fixation or are lost entirely by chance. 

For any set of gene copies at present (time = 0), we can trace their genealogies back to a common ancestor. 
    </p>
</div>

In [27]:
wright_fisher_simulation(
    time=31, popsize=11, sort_edges=True, seed=123, allele_frequency=0.5, 
    node_size=7, height=600, nsamples=6,
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="600.0px" viewBox="0 0 500.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbbc2fbb84aa9497f8537999af4658d20"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);str

<div class="alert alert-block alert-success">
    <h3>What are the assumptions of the Wright Fisher model?</h3>
    <ol>
      <li>Panmixia (random mating)</li>
      <li>Hermaphroditic organisms</li>
      <li>No mutation</li>
      <li>No migration / no gene flow</li>
      <li>No selection</li>
    </ol>
</div>


<div class="alert alert-block alert-info">
    <h3>Action: How do these assumptions affect the applicability of the Wright Fisher Model?</h3>
    <p>
        Look at the list of assumptions and try to think about what kind of limitations they present for using the Wright Fisher Model to try to understand evolution. What can we learn, and what can't we learn? 
        
Jot down some thoughts in the code block below
    </p>
</div>

[double-click to edit]

## Revisit: Coalescent Model

The coalescent process uses the assumptions of the Wright Fisher model. It works by starting with a population of genes copies at present, and calculating parameters of interest based on probabilities of coalescent events going backwards in time. 

`ipcoal` is a python wrapper around the coalescent simulator `msprime`, which we will use to simulate coalescent trees. 

The only parameter we need to set for a coalescent model is `Ne` - the *effective population size*. 

Because of the assumptions of the model, there are no other parameters to set. The model will run until all of the gene copies in the `Ne` number of indivudals coalesce into a single ancestor. 

Below we will simulate genealogies for 10 individuals in a population size of 2000.

`ipcoal` simulates recombination by default. To simulate a standard WF model, we have to specify a recombination rate of 0.0.

In [28]:
# seed for random number generator (RNG)
#SEED = 32985743

NE = 2000
NSAMPLES = 10
MUT = 1e-6

In [29]:
# setup simulation with small Ne
model1 = ipcoal.Model(
    Ne=NE, 
    nsamples=NSAMPLES,
    recomb=0.0, 
    mut=MUT,
    store_tree_sequences=True, 
    #seed_mutations=SEED, 
    #seed_trees=SEED,
)

# simulate one short uninformative locus
model1.sim_loci(nloci=1, nsites=100)

In [30]:
#plot the tree
model1.draw_genealogy(layout='d', show_substitutions=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2686b7a824774cd8aa404019e6482597"> p_2 p_9 p_1 p_3 p_0 p_8 p_5 p_4 p_6 p_7 id: 0
site: 0
time: 3782
mtype: 0 id: 1
site: 1
time: 1043
mtype: 0 tree in interval 0 (sites 0-100)

<div class="alert alert-block alert-success">
    <p>
The code above plots a single genealogy with the substitutions that occurred during the simulation. 
        
We can also look the topology of the trees generated with the same starting parameters
</p>
</div>

In [47]:
# generate 12 random trees with same params as above
trees = []
for i in range(12):
    model = ipcoal.Model(
        Ne=NE, 
        nsamples=NSAMPLES,
        recomb=0, 
        mut=MUT,
        store_tree_sequences=True,
    )
    
    model.sim_loci(nloci=1, nsites=100)
    
    trees.append(toytree.tree(model.df.genealogy[0]))
    tmrcas = [i.treenode.height for i in trees]

mtree = toytree.mtree(trees)

#Report average tMRCA
print(f"The estimated tMRCA in a population with Ne={NE} is: {np.mean(tmrcas):.2f}, "
      f"with standard deviation: {np.std(tmrcas):.2f}")

# draw all 12 trees
mtree.draw(shape=(3, 4), shared_axes=True, scale_bar=True, layout='d', height=600);

The estimated tMRCA in a population with Ne=2000 is: 7381.89, with standard deviation: 4640.29


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="600.0px" viewBox="0 0 750.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbfb9e89f19bf48af9e6309f7c2bbe0c3"> p_7 p_0 p_2 p_5 p_6 p_8 p_9 p_1 p_3 p_4 0 8000 16000 p_7 p_2 p_8 p_9 p_0 p_4 p_3 p_5 p_1 p_6 0 8000 16000 p_7 p_2 p_1 p_3 p_0 p_5 p_8 p_9 p_4 p_6 0 8000 16000 p_3 p_7 p_8 p_0 p_5 p_9 p_6 p_2 p_1 p_4 0 8000 16000 p_4 p_7 p_9 p_1 p_2 p_0 p_6 p_8 p_3 p_5 0 8000 16000 p_1 p_4 p_3 p_5 p_7 p_9 p_0 p_2 p_6 p_8 0 8000 16000 p_2 p_7 p_1 p_8 p_9 p_3 p_5 p_6 p_0 p_4 0 8000 16000 p_6 p_7 p_4 p_5 p_0 p_1 p_3 p_8 p_2 p_9 0 8000 16000 p_9 p_3 p_0 p_6 p_2 p_4 p_5 p_8 p_1 p_7 0 8000 16000 p_7 p_5 p_0 p_1 p_8 p_3 p_2 p_4 p_6 p_9 0 8000 16000 p_3 p_2 p_5 p_4 p_7 p_0 p_6 p_9 p_1 p_8 0 8000 16000 p_0 p_8 p_7 p_4 p_9 p_1 p_2 p_6 p_3 p_5 0 8000 16000

<div class="alert alert-block alert-info">
    <h3>Action: Re-run the code block again a couple of times to see the different coalescent outcomes</h3>
    <p>
        <b>How do the trees compare to each other?</b><br><br>
        Describe any observations in the code block below.
    </p>
</div>


[double-click to edit]

<div class="alert alert-block alert-success">
    <p>
        The coalescent process is stochastic, the results will be different every time you run the model, even for the same population size. 
    </p>
</div>

<div class="alert alert-block alert-info">
    <h3>Action: What kinds of things can we learn from coalescent simulations?</h3>
    <p>
        Jot down some thoughts in the code block below
    </p>
</div>

[double-click to edit]

## Forward-Time Simulations using SLiM shadie
SLiM is a sophisticated forward-time simulator developed by the Messer lab at Cornell. SLiM can do many things, but the learning curve for new users is steep. SLiM uses its own programming language, called Eidos, and the manual is ~650 pages long. 

We wrote a python wrapper around SLiM to make some of the SLiM functionality available in a python framework. We also developed this program to investigate patterns of genome evolution in plants with alternation of generations life cycles. Our program is called `shadie`, which stands for "Simulating Haploid Diploid Evolution". 

Below, we will use `shadie` to compare forward-time simulations to coalescent (also called 'backward-time') simulations and compare the advantages and disadvantages of each approach

### Making the Chromosome
SLiM takes an explicit chromosome model, which is where all the features of the genome will be defined. This is the first stark contrast to coalescent simulations, in which the only thing you need to define about the genome is the length, because all the sites are assumed to be neutral. 

`shadie` makes it easy to design a chromosome by giving the user tools to generate and visualize the chromosome model. For now, we want to see how forward-time simulations compare to coalescent, so we need to make a completely neutral chromosome that is equivalent to the one we made for our coalescent simulations. 

We used a coalescent model with a single locus and 100 sites, so for SLiM we will use `shadie` to build a "neutral" chromsome that is 100 base-pairs long. 

In [36]:
neutral_chrom = shadie.chromosome.explicit({
    (0, 99): shadie.NONCDS,
})

In [37]:
neutral_chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdfd060b5cb694799b049bb573f5c2fce"> name: noncds
interval: (0, 99)
ElementType: g3
coding: False 0 10 20 30 40 50 60 70 80 90

This is just one solid block of neutral chromosome, so it doens't look like much. We can see a little more using `shadie's` interactive chromosome viewer:

In [38]:
neutral_chrom.inspect()

alt.VConcatChart(...)

<div class="alert alert-block alert-success">
    <p>
        Now, we will set up simulations in shadie with the same parameters as our coalescent simulations above. 
    </p>
</div>

In [18]:
with shadie.Model() as slim_model:
    slim_model.initialize(
        chromosome=neutral_chrom, 
        sim_time=10000,
        mutation_rate=MUT, 
        recomb_rate=0.0,
        file_out="../data/wf.trees",
    )

    slim_model.reproduction.wright_fisher(NE)

<div class="alert alert-block alert-success">
    <p>
You can uncomment the code block below to view the Eidos script that will run the simulation in SLiM. 

If you have installed SLiM (https://messerlab.org/slim/) and want to see the simulation run in the SLiM gui, you can simply copy and paste this script into the SLiM gui and run it. You will also be able to edit any of the simulation parameters and see how they affect the simulation as it runs in real time.
</p>
</div>

In [19]:
#print(slim_model.script)

In [20]:
%%time
slim_model.run(np.random.randint(2**31))

CPU times: user 3.61 ms, sys: 20.3 ms, total: 23.9 ms
Wall time: 52.2 s


<div class="alert alert-block alert-success">
    <p>
The code block above reports the time it took for the simulation to finish (Wall time). How long did that simulation take compared to the coalescent one? 

Now we will use another program called tskit to load the SLiM file output, which is a `.trees` file. 
</p>
</div>

In [39]:
postsim = tskit.load("../data/wf.trees")
postsim

In [44]:
tts = toytree_sequence(postsim, sample=10)

tts.draw_tree(0, layout = 'd', scale_bar = True,);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="270.0px" viewBox="0 0 350.0 270.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t214ba0a32ea747cbb66d4cd6068aabf8"> p0-1 p0-4 p0-0 p0-6 p0-7 p0-5 p0-9 p0-3 p0-2 p0-8 id: 0
site: 0
time: 1801
mtype: 1 0 5000 10000 tree at site 0

<div class="alert alert-block alert-info">
    <h3>Compare</h3>
    <p>How does this compare to the tree from the coalescent process? Why is there a long root after ~8000 generations? 
    
One way the forward-time simulation is different from the coalescent is that it explicitly simulates all 4000 chromosomes in all 2000 individuals each generation of the simulation. This is part of what makes it slower, but it also means it generates more data. Instead of just simulating the 10 chromosomes sampled at present, as the coalescent does, the SLiM simulation has simulated all 4000 chromosomes in the entire population at present. The code above chooses 10 of those chromosomes at random and plots their genealogy. Try re-running the code block above to see how the genealogies between different chromosomes change. 
    </p>
</div>

[double-click to edit]

<div class="alert alert-block alert-info">
    <h3>Speculate: Burn-In</h3>
    <p>For computational reasons, the default behavior in SLiM is to use a "blank" chromosome, to which mutations are added. So, instead of starting with a string of As, Ts, Cs, and Gs, the chromosome is just a big empty container. 
        
In order for us to get a realistic behavior from the model, we need to run a 'neutral burn-in'. The minimum 'acceptable' duration for this burn-in is `4*Ne` generations. 

*Why do you think we need this burn-in? Why is `4*Ne` a good length of time for the burn-in?* Speculate in the cell below
        
We will revisit this question later. </p>
</div>

[double-click to edit]

<div class="alert alert-block alert-danger">
    <p>
This next cell will run 12 neutral shadie simulations - this should take about 15 minutes. Change the block below from a markdown to a code block in order to run it (this is to keep you from running it by accident). 
        
The only change from above is that we have added an extra line with the `file-in` parameter for the burn-in. This instructs the SLiM simulation to read in the burn-in from a finished simulation file that was previously run. The code for the burn-in is at the end of this notebook in the Appendix. 
    </p>
</div>

shadie_trees = []

for i in range(12):
    FILEOUT = f"../data/wf{i}.trees"
    
    with shadie.Model() as model:
        model.initialize(
            chromosome=neutral_chrom, 
            sim_time=10000,
            mutation_rate=MUT, 
            recomb_rate=0,
            file_in = "../data/wf_burn.trees",
            file_out=FILEOUT,
        )

        model.reproduction.wright_fisher(NE)
    
    model.run(np.random.randint(2**31))
    
    postsim = tskit.load(FILEOUT)
    tts = toytree_sequence(postsim, sample=10)
    shadie_trees.append(toytree.tree(tts.at_site(0).get_nodes()[-2]))
    #shadie_trees.append(tts.at_site(0))

<div class="alert alert-block alert-danger">
    <p>
If you don't want to wait for the simulations to finish, we have provided some pre-run files in the binder. You can simply load the trees from the filesusing the code below.

(<b>Note</b>: change the block from a markdown block to a code block in order to run it).
    </p>
</div>


shadie_trees = []
for i in range(12):
    FILEOUT = f"../data/wf{i}.trees"
    postsim = tskit.load(FILEOUT)
    
    tts = toytree_sequence(postsim, sample=10)
    
    shadie_trees.append(toytree.tree(tts.at_site(0).get_nodes()[-2]))

<div class="alert alert-block alert-success">
    <p>
In order to calculate the tMRCAs from a forward simulation, we have to trim the root off the end of the simulation (during which time all the chromosomes have coalesced). The code above takes care of this. 
    </p>
</div>

In [46]:
tmrcas = [i.treenode.height for i in shadie_trees]
shadie_mtree = toytree.mtree(shadie_trees)

#Report average tMRCA
print(f"The estimated tMRCA in a population with Ne={NE} is: {np.mean(tmrcas):.2f}, "
      f"with standard deviation: {np.std(tmrcas):.2f}")

# draw all 12 trees
shadie_mtree.draw(shape=(3, 4), shared_axes=True, scale_bar=True, layout='d', height=600);

The estimated tMRCA in a population with Ne=2000 is: 6643.67, with standard deviation: 4780.64


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="750.0px" height="600.0px" viewBox="0 0 750.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t654df82ed0c440009cdb22c15612ddd3"> p0-8 p0-0 p0-6 p0-3 p0-7 p0-5 p0-2 p0-1 p0-4 p0-9 0 8000 16000 p0-1 p0-7 p0-2 p0-3 p0-6 p0-4 p0-9 p0-8 p0-0 p0-5 0 8000 16000 p0-0 p0-3 p0-7 p0-9 p0-1 p0-4 p0-2 p0-6 p0-5 p0-8 0 8000 16000 p0-7 p0-0 p0-4 p0-2 p0-8 p0-1 p0-3 p0-6 p0-5 p0-9 0 8000 16000 p0-0 p0-4 p0-6 p0-9 p0-1 p0-2 p0-8 p0-7 p0-3 p0-5 0 8000 16000 p0-1 p0-7 p0-0 p0-2 p0-4 p0-5 p0-8 p0-3 p0-6 p0-9 0 8000 16000 p0-9 p0-0 p0-5 p0-8 p0-3 p0-4 p0-6 p0-1 p0-2 p0-7 0 8000 16000 p0-2 p0-7 p0-4 p0-6 p0-5 p0-8 p0-1 p0-0 p0-3 p0-9 0 8000 16000 p0-1 p0-6 p0-3 p0-5 p0-9 p0-4 p0-0 p0-8 p0-2 p0-7 0 8000 16000 p0-4 p0-0 p0-7 p0-9 p0-2 p0-5 p0-8 p0-1 p0-3 p0-6 0 8000 16000 p0-8 p0-7 p0-2 p0-3 p0-5 p0-1 p0-4 p0-9 p0-0 p0-6 0 8000 16000 p0-4 p0-5 p0-0 p0-3 p0-6 p0-1 p0-2 p0-7 p0-8 p0-9 0 8000 16000

<div class="alert alert-block alert-info">
    <h3>Review: Comparison</h3>
    <p>How do these trees compare to the coalescent trees? Is this what you expected to see based on the coalescent simulations? Compared the average tMRCA and the standard deviation.</p>
</div>

[double-click to edit]

<div class="alert alert-block alert-success">
    <h2>Forward vs. coalescent simulations under pure Wright-Fisher process</h2>
    <p>Now, let's check that the forward sims give us something similar to the coalescent when we simulate many loci and look at population diversity</p>
</div>

In [48]:
# set variables
Ne = 5000
nsamples = 8
mut = 1e-7
nloci = 1000
loclen = 20

<div class="alert alert-block alert-success">
    <p>How would we make this chromosome? We need 1000 loci, each of length 20. Because the loci are unlinked, there is no recombination between them and we can model that in SLiM as just a solid block of 20*1000 neutral base pairs</p>
</div>

In [49]:
length = nloci*loclen

long_chrom = shadie.chromosome.explicit({
    (0, length): shadie.NONCDS,
})
long_chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2491d38c673e4b948f1d05783da8da48"> name: noncds
interval: (0, 20000)
ElementType: g3
coding: False 0 5000 10000 15000 20000

<div class="alert alert-block alert-danger">
    <p>The code block below will run the simulation. This is a more complex simulation, and will take about 15 min to run. Again, change the block from Markdown to code in order to run it.</p>
</div>

%%time
with shadie.Model() as slim_theta_model:
    slim_theta_model.initialize(
        chromosome=neutral_chrom, 
        sim_time=10000,
        mutation_rate=mut, 
        recomb_rate=0,
        file_in = "wf_mburn.trees", #code in appendix
        file_out="wf_mloci.trees",
    )

    slim_theta_model.reproduction.wright_fisher(Ne)
slim_theta_model.run(np.random.randint(2**31))

In [50]:
load_ts = tskit.load("../data/wf_mloci.trees")
ts_mloci = load_ts.simplify() #this is necessary for technicalities related to the burn-in
ts_mloci

<div class="alert alert-block alert-success">
    <p>Now, let's run the coalescent simulation</p>
</div>

In [51]:
# simulate sequence data for a single population at many loci
model = ipcoal.Model(Ne=Ne, nsamples=nsamples, mut=mut)
model.sim_loci(nloci=nloci, nsites=loclen)

# combine the loci into one large sequence matrix
model.seqs = np.concatenate(model.seqs, 1)

<div class="alert alert-block alert-success">
    <p>That was <i>way</i> faster than the forward simulation - and we even saved time on our forward sim by loading the burn-in .
    
Let's double check that the population diversity is what we expect.</p>
</div>

In [52]:
def get_pop_gen_diversity(seqs):
    """Return the proportion diffs measured for every pair of samples."""
    nsamples = seqs.shape[0]
    
    # sample all combination of two gene copies
    samples = list(itertools.combinations_with_replacement(range(nsamples), 2))

    # measure proportion of variable sites in each sample of two gene copies
    theta = []
    idx = 0
    for sample in samples:
        i, j = sample
        if i != j:
            two_samples = seqs[[i, j], :]
            theta.append((sum(two_samples[1, :] != two_samples[0, :]) / two_samples.shape[1]))
            idx += 1
    return theta

In [53]:
# report results
theta = get_pop_gen_diversity(model.seqs)
print("measured population genetic diversity (theta)")
print('mean={:.4f}, std={:.4f}'.format(np.mean(theta), np.std(theta)))
print("\ntheoretical expectation (4Neu): {}".format(4 * Ne * mut))

measured population genetic diversity (theta)
mean=0.0016, std=0.0002

theoretical expectation (4Neu): 0.002


<div class="alert alert-block alert-success">
    <p>The code below uses a custom script in shadie to calculate theta on the tree sequence file from the forward-time SLiM simulation.</p>
</div>

In [3]:
stats(ts_mloci)

,mean,CI_5%,CI_95%
theta,0.002084,0.001980,0.002188
D_Taj,-0.438024,-0.569243,-0.306806


<div class="alert alert-block alert-success">
<h3>Success!</h3>
    <p>Theta for our forward sim matches our theoretical expectation, and the coalescent simulations.</p>
</div>

<div class="alert alert-block alert-info">
<h3>What are the limitations of the coalescent?</h3>
    <p>The coalescent may be fast, but can you summarize what some of the limitations are, based on these examples and the assumptions made by the coalescent?
    </p>
</div>

[double-click to edit]

<div class="alert alert-block alert-warning">
    <h2>Let's add in some complexity</h2>
    <p>Because of the simplifying assumptions made by the coalescent model it is mathematically tractable - and this is what makes it so fast. As those assumptions are relaxed, the math become increasingly difficult.

Foward simulation shines when we start adding complexity, like recombination and selected sites. Let's relax some of the assumptions of the coalescent and see what happens. 
</p>
</div>

In [54]:
#This chromosome has introns, exons, and non-coding regions
default_chrom = shadie.chromosome.default()

default_chrom.inspect()

alt.VConcatChart(...)

You may be curious what it means for part of the chromosome to be an "exon" or an "intron". `shadie` has default exons and introns, with their parameters already set. However, the use is able to make their own chromosome regions and specify their fitness effects however the want. We won't get into that kind of detail here, but if you're curious you can check out the [shadie docs](https://elissasoroj.github.io/shadie/notebooks/2-chrom/) for more info. 

In brief, the different regions, such as 'exon' and 'intron' will experience mutations with probability equal to the mutation rate each generation. The fitness effect of each mutation will be drawn from a fitness distribution, which is defiend by the user. `shadie` has convenience functions that help visualize the distributions that fitness effects are drawn from. 

Below you can see that the default exon in `shadie` mostly experiences deleterious mutations, but a tiny part of the distribution includes positive effects (i.e. beneficial mutations), whereas the intron can only experience deleterious mutations. 

In [55]:
shadie.EXON.draw();
shadie.INTRON.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="250.0px" viewBox="0 0 350.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t09d73794fc4543f396f3980b85c33cd9"> <polygon points="50.0,200.0 50.501002004008,200.0 51.002004008016044,200.0 51.503006012024045,200.0 52.00400801603205,200.0 52.505010020040096,200.0 53.0060120240481,200.0 53.5070140280561,200.0 54.00801603206414,200.0 54.50901803607214,200.0 55.01002004008015,200.0 55.51102204408819,200.0 56.012024048096194,200.0 56.5130260521042,200.0 57.01402805611224,200.0 57.51503006012024,200.0 58.01603206412825,200.0 58.51703406813629,200.0 59.01803607214429,200.0 59.5190380761523,200.0 60.02004008016033,200.0 60.521042084168336,200.0 61.022044088176344,200.0 61.52304609218439,200.0 62.02404809619239,200.0 62.525050100200396,200.0 63.02605210420843,200.0 63.52705410821643,200.0 64.02805611222445,200.0 64.52905811623248,200.0 65.03006012024048,200.0 65.53106212424849,200.0 66.03206412825652,200.0 66.53306613226454,200.0 67.03406813627254,200.0 67.53507014028057,200.0 68.03607214428858,200.0 68.5370741482966,200.0 69.0380761523046,200.0 69.53907815631263,200.0 70.04008016032063,200.0 70.54108216432863,200.0 71.04208416833667,200.0 71.54308617234469,200.0 72.04408817635269,200.0 72.54509018036073,200.0 73.04609218436873,200.0 73.54709418837673,200.0 74.04809619238478,200.0 74.54909819639278,200.0 75.05010020040078,200.0 75.55110220440882,200.0 76.05210420841684,200.0 76.55310621242484,200.0 77.05410821643287,200.0 77.55511022044087,200.0 78.05611222444888,200.0 78.55711422845692,200.0 79.05811623246493,200.0 79.55911823647293,200.0 80.06012024048097,200.0 80.56112224448897,200.0 81.06212424849699,200.0 81.56312625250501,200.0 82.06412825651302,200.0 82.56513026052103,200.0 83.06613226452906,200.0 83.56713426853706,200.0 84.06813627254508,200.0 84.56913827655312,200.0 85.0701402805611,200.0 85.57114228456913,200.0 86.07214428857716,200.0 86.57314629258516,200.0 87.07414829659317,200.0 87.57515030060121,200.0 88.07615230460922,200.0 88.57715430861722,200.0 89.07815631262525,200.0 89.57915831663325,200.0 90.08016032064127,200.0 90.5811623246493,200.0 91.08216432865731,200.0 91.58316633266531,200.0 92.08416833667334,200.0 92.58517034068136,200.0 93.08617234468937,200.0 93.5871743486974,200.0 94.0881763527054,200.0 94.5891783567134,200.0 95.09018036072146,200.0 95.59118236472946,200.0 96.09218436873746,200.0 96.59318637274549,200.0 97.09418837675351,200.0 97.5951903807615,200.0 98.09619238476955,200.0 98.59719438877755,200.0 99.09819639278555,200.0 99.59919839679358,200.0 100.1002004008016,200.0 100.60120240480961,200.0 101.10220440881764,200.0 101.60320641282564,200.0 102.10420841683366,200.0 102.6052104208417,200.0 103.1062124248497,200.0 103.6072144288577,200.0 104.10821643286575,200.0 104.60921843687375,200.0 105.11022044088176,200.0 105.61122244488979,200.0 106.11222444889779,200.0 106.61322645290579,200.0 107.11422845691382,200.0 107.61523046092185,200.0 108.11623246492985,200.0 108.61723446893788,200.0 109.11823647294588,200.0 109.6192384769539,200.0 110.12024048096194,200.0 110.62124248496994,200.0 111.12224448897794,200.0 111.62324649298598,200.0 112.124248496994,200.0 112.625250501002,200.0 113.12625250501003,200.0 113.62725450901803,200.0 114.12825651302605,200.0 114.62925851703409,200.0 115.13026052104208,200.0 115.6312625250501,200.0 116.13226452905812,200.0 116.63326653306613,200.0 117.13426853707412,200.0 117.63527054108218,200.0 118.13627254509018,200.0 118.63727454909818,200.0 119.13827655310624,200.0 119.63927855711424,200.0 120.14028056112222,200.0 120.64128256513027,200.0 121.142284569138

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="250.0px" viewBox="0 0 350.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="taa9df20a791f44cdbb5c5faf020a8286"> <polygon points="50.0,200.0 50.50100200400803,200.0 51.00200400801602,200.0 51.50300601202406,200.0 52.00400801603206,200.0 52.50501002004009,200.0 53.00601202404808,200.0 53.50701402805612,200.0 54.00801603206412,200.0 54.50901803607214,200.0 55.010020040080185,200.0 55.51102204408818,200.0 56.01202404809621,200.0 56.5130260521042,200.0 57.01402805611224,200.0 57.51503006012023,200.0 58.01603206412826,200.0 58.51703406813626,200.0 59.01803607214429,200.0 59.51903807615229,200.0 60.02004008016032,200.0 60.52104208416835,200.0 61.022044088176344,200.0 61.52304609218439,200.0 62.02404809619239,200.0 62.52505010020041,200.0 63.02605210420841,200.0 63.52705410821643,200.0 64.02805611222443,200.0 64.52905811623248,200.0 65.03006012024046,200.0 65.5310621242485,200.0 66.03206412825652,200.0 66.53306613226452,200.0 67.03406813627255,200.0 67.53507014028055,200.0 68.03607214428858,200.0 68.53707414829658,200.0 69.03807615230461,200.0 69.53907815631261,200.0 70.04008016032064,200.0 70.54108216432863,200.0 71.04208416833667,200.0 71.5430861723447,200.0 72.0440881763527,200.0 72.54509018036073,200.0 73.04609218436873,200.0 73.54709418837676,200.0 74.04809619238475,200.0 74.54909819639279,200.0 75.05010020040082,200.0 75.55110220440882,200.0 76.05210420841682,200.0 76.55310621242484,200.0 77.05410821643287,200.0 77.55511022044087,200.0 78.05611222444891,200.0 78.5571142284569,200.0 79.05811623246494,200.0 79.55911823647293,200.0 80.06012024048096,200.0 80.56112224448898,200.0 81.06212424849699,200.0 81.56312625250501,200.0 82.06412825651302,200.0 82.56513026052106,200.0 83.06613226452905,200.0 83.56713426853707,200.0 84.06813627254508,200.0 84.56913827655312,200.0 85.0701402805611,200.0 85.57114228456913,200.0 86.07214428857716,200.0 86.57314629258516,200.0 87.0741482965932,200.0 87.5751503006012,200.0 88.07615230460922,200.0 88.57715430861722,200.0 89.07815631262525,200.0 89.57915831663325,200.0 90.08016032064128,200.0 90.58116232464928,200.0 91.08216432865731,200.0 91.58316633266534,200.0 92.08416833667334,200.0 92.58517034068137,200.0 93.08617234468937,200.0 93.5871743486974,200.0 94.0881763527054,200.0 94.58917835671343,200.0 95.09018036072146,200.0 95.59118236472946,200.0 96.09218436873746,200.0 96.59318637274549,200.0 97.09418837675352,200.0 97.59519038076152,200.0 98.09619238476955,200.0 98.59719438877755,200.0 99.09819639278558,200.0 99.59919839679358,200.0 100.10020040080161,200.0 100.60120240480964,200.0 101.10220440881764,200.0 101.60320641282564,200.0 102.10420841683367,200.0 102.6052104208417,200.0 103.10621242484969,200.0 103.60721442885773,200.0 104.10821643286572,200.0 104.60921843687376,200.0 105.11022044088176,200.0 105.61122244488978,200.0 106.11222444889782,200.0 106.6132264529058,200.0 107.1142284569138,200.0 107.61523046092185,200.0 108.11623246492988,200.0 108.61723446893787,200.0 109.1182364729459,200.0 109.6192384769539,200.0 110.12024048096193,200.0 110.62124248496993,200.0 111.12224448897797,200.0 111.62324649298598,200.0 112.124248496994,200.0 112.62525050100197,200.0 113.12625250501,200.0 113.62725450901804,200.0 114.12825651302605,200.0 114.62925851703409,200.0 115.13026052104206,200.0 115.6312625250501,200.0 116.13226452905809,200.0 116.63326653306613,200.0 117.13426853707415,200.0 117.63527054108216,200.0 118.13627254509021,200.0 118.63727454909821,200.0 119.13827655310624,200.0 119.63927855711421,200.0 120.14028056112224,200.0 120.64128256513027,200.0 121.14228456913828

<div class="alert alert-block alert-success">
    <h3>Set up the simulation</h3>

Playing around with designing chromosomes can be fun - feel free to make some extra code blocks and try messing around the `shadie's` chromosome module. `shadie.chromosome.random()` is a good place to start. Make sure to check out the docs for more guidance. 
        
Now, we want to set up a selected simulation. We're going to do something clever here and use the same burn-in that we used for the neutral simulation, but start a selected simulation from that point of neutral variation. SLiM is designed to do this, and shadie is suppoed to make it easy. The only the that needs to stay the same across the simulations is the chromosome length. 
        
The reason we want to use a burn-in is because we want to start with realistic "standing variation" in our simulated population. If we started with all the individuals having empty chromosomes, that's essentially the same as starting out with a population of completely homozygous clones. The initial evolutionary trajectory of that population will be vastly different from the kind of population we usually observe in nature. The reason `4Ne` is the minimum amount of time a burn-in should run for is because that is the expected time for all the chromosomes in all the individuals to coalesce. 

</div>

In [56]:
sel_chrom = shadie.chromosome.explicit({
    (0, 6999): shadie.NONCDS,
    (7000, 8999): shadie.INTRON,
    (9000, 10999): shadie.EXON,
    (11000, 12999): shadie.INTRON,
    (13000, 20000): shadie.NONCDS,
})
sel_chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4d6d47ca767c465ab54d1231abaff042"> name: noncds
interval: (0, 6999)
ElementType: g3
coding: False name: intron
interval: (7000, 8999)
ElementType: g2
coding: True name: exon
interval: (9000, 10999)
ElementType: g1
coding: True name: intron
interval: (11000, 12999)
ElementType: g2
coding: True name: noncds
interval: (13000, 20000)
ElementType: g3
coding: False 0 5000 10000 15000 20000

In [195]:
with shadie.Model() as model_selection:
    model_selection.initialize(
        chromosome=sel_chrom, 
        sim_time=10000,
        mutation_rate=mut, 
        recomb_rate=1e-8,
        file_in="../data/wf_mburn.trees",
        file_out="../data/selection2.trees",
    )
    model_selection.reproduction.wright_fisher(Ne)
    
#print(model_selection.script)

<div class="alert alert-block alert-danger">
    <p>The code block below will run the simulation. Again, change the block from Markdown to code in order to run it.</p>
</div>

model_selection.run(np.random.randint(2**31))

In [57]:
ts = tskit.load("../data/selection.trees")
sel_ts = ts.simplify()

In [58]:
sel_ts

In [59]:
stats(sel_ts)

,mean,CI_5%,CI_95%
theta,0.000076,0.000070,0.000081
D_Taj,-1.500427,-1.574403,-1.426451


<div class="alert alert-block alert-info">
<h3>Compare</h3>
    <p>What happened to the overall diversity in this population compared to the neutral simulation? Why do you think selection has this effect?
    </p>
</div>

[double-click to edit]

<div class="alert alert-block alert-success">
<h3>How do we analyze genomic data?</h3>
    <p>The function below uses a custom script from shadie to plot diversity in windows across the genome (x-axis is in the unit "window".

Let's first look at the neutral simulation. What does diversity look like in windows across the genomw? 
</p>
</div>

In [60]:
draw_stats(ts_mloci);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdab706e81ad5470c9d384a0a6ca56d13"> <polygon points="80.0,250.0 89.73684210526315,250.0 99.47368421052633,250.0 109.21052631578947,250.0 118.94736842105263,250.0 128.68421052631578,250.0 138.42105263157896,250.0 148.15789473684208,250.0 157.89473684210526,250.0 167.6315789473684,250.0 177.3684210526316,250.0 187.10526315789474,250.0 196.8421052631579,250.0 206.57894736842104,250.0 216.31578947368422,250.0 226.05263157894734,250.0 235.78947368421052,250.0 245.5263157894737,250.0 255.26315789473682,250.0 265.0,250.0 274.7368421052632,250.0 284.47368421052636,250.0 294.2105263157895,250.0 303.9473684210526,250.0 313.6842105263158,250.0 323.42105263157896,250.0 333.1578947368421,250.0 342.89473684210526,250.0 352.63157894736844,250.0 362.36842105263156,250.0 372.10526315789474,250.0 381.8421052631579,250.0 391.57894736842104,250.0 401.3157894736842,250.0 411.0526315789474,250.0 420.7894736842105,250.0 430.52631578947364,250.0 440.2631578947369,250.0 450.0,250.0 450.0,187.47620500305112 440.2631578947369,195.99084807809646 430.52631578947364,60.0 420.7894736842105,178.79926784624826 411.0526315789474,228.7684563758389 401.3157894736842,195.79377669310594 391.57894736842104,155.1159243441123 381.8421052631579,175.26937156802964 372.10526315789474,248.26113483831605 362.36842105263156,216.3993288590604 352.63157894736844,170.1455155582676 342.89473684210526,185.18669920683365 333.1578947368421,191.24954240390483 323.42105263157896,178.43990237949996 313.6842105263158,212.81726662599164 303.9473684210526,171.24100061012848 294.2105263157895,185.62721171446023 284.47368421052636,175.70408785845027 274.7368421052632,193.22025625381346 265.0,165.3114704087862 255.26315789473682,194.52440512507633 245.5263157894737,230.5131177547286 235.78947368421052,166.86485661989053 226.05263157894734,186.33435021354512 216.31578947368422,241.50274557657107 206.57894736842104,127.36943258084227 196.8421052631579,160.7846247712023 187.10526315789474,154.00884685784052 177.3684210526316,145.58694325808437 167.6315789473684,190.83801098230637 157.89473684210526,162.17571690054947 148.15789473684208,195.89231238560114 138.42105263157896,248.25533862111044 128.68421052631578,207.41519219036013 118.94736842105263,185.45332519829182 109.21052631578947,157.5329469188531 99.47368421052633,165.54911531421624 89.73684210526315,204.78370957901166 80.0,170.58602806589406" style="fill:rgb(12.5%,69.8%,66.7%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.000 0.001 0.002 0.003 0.004 0.005 0.006 diversity in 500bp windows

<div class="alert alert-block alert-success">
    <p>This is a crude plot because our chromosome was quite small. However, we can generally deduce that there is some standing diversity across the neutral chromosome. And, we can start to see a pattern when we compare it to the simulation with selection:</p>
</div>

In [61]:
draw_stats(sel_ts);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta583832e5a2c4d9292bc7b20e7fc28cd"> <polygon points="80.0,250.0 89.73684210526315,250.0 99.47368421052633,250.0 109.21052631578947,250.0 118.94736842105263,250.0 128.68421052631578,250.0 138.42105263157896,250.0 148.15789473684208,250.0 157.89473684210526,250.0 167.6315789473684,250.0 177.3684210526316,250.0 187.10526315789474,250.0 196.8421052631579,250.0 206.57894736842104,250.0 216.31578947368422,250.0 226.05263157894734,250.0 235.78947368421052,250.0 245.5263157894737,250.0 255.26315789473682,250.0 265.0,250.0 274.7368421052632,250.0 284.47368421052636,250.0 294.2105263157895,250.0 303.9473684210526,250.0 313.6842105263158,250.0 323.42105263157896,250.0 333.1578947368421,250.0 342.89473684210526,250.0 352.63157894736844,250.0 362.36842105263156,250.0 372.10526315789474,250.0 381.8421052631579,250.0 391.57894736842104,250.0 401.3157894736842,250.0 411.0526315789474,250.0 420.7894736842105,250.0 430.52631578947364,250.0 440.2631578947369,250.0 450.0,250.0 450.0,248.94486314954383 440.2631578947369,215.557318524395 430.52631578947364,190.2340341134468 420.7894736842105,245.25188417294726 411.0526315789474,231.08290360967865 401.3157894736842,205.38278460928228 391.57894736842104,239.4486314954383 381.8421052631579,241.0313367711226 372.10526315789474,150.21420071400243 362.36842105263156,219.17493058310185 352.63157894736844,234.2483141610472 342.89473684210526,179.53193177310604 333.1578947368421,245.7794525981753 323.42105263157896,241.0313367711226 313.6842105263158,249.47243157477192 303.9473684210526,249.47243157477192 294.2105263157895,250.0 284.47368421052636,250.0 274.7368421052632,246.8345894486315 265.0,60.0 255.26315789473682,246.3070210234034 245.5263157894737,250.0 235.78947368421052,250.0 226.05263157894734,248.94486314954383 216.31578947368422,250.0 206.57894736842104,172.59817532725083 196.8421052631579,214.42681475604923 187.10526315789474,178.62752875842926 177.3684210526316,170.4879016263385 167.6315789473684,239.52399841332806 157.89473684210526,233.79611265370883 148.15789473684208,215.029750099167 138.42105263157896,244.19674732249106 128.68421052631578,207.49305831019439 118.94736842105263,175.16065053550182 109.21052631578947,223.3954779849266 99.47368421052633,218.7227290757636 89.73684210526315,129.78976596588643 80.0,240.50376834589449" style="fill:rgb(12.5%,69.8%,66.7%);fill-opacity:1.0;opacity:0.5;stroke:none"> 0 10 20 30 0.00000 0.00005 0.00010 0.00015 0.00020 0.00025 0.00030 0.00035 0.00040 0.00045 diversity in 500bp windows

<div class="alert alert-block alert-success">
    <p>What do you think is happening? Recall the chromosome model for this simulation:</p>
</div>

In [62]:
sel_chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t449dbcc765d44acea613e3569dbc3b72"> name: noncds
interval: (0, 6999)
ElementType: g3
coding: False name: intron
interval: (7000, 8999)
ElementType: g2
coding: True name: exon
interval: (9000, 10999)
ElementType: g1
coding: True name: intron
interval: (11000, 12999)
ElementType: g2
coding: True name: noncds
interval: (13000, 20000)
ElementType: g3
coding: False 0 5000 10000 15000 20000

<div class="alert alert-block alert-info">
    <p>Window #20 corresponds approximately to 10,000 bp on the chromosome. What's happening to the diversity on the plot at this point? What do you think the spike in diverity at window #19 might indicate?</p>
</div>

[double-click to edit]

<div class="alert alert-block alert-warning">
    <h2>Caveats</h2>
    <p>This was just a tiny taste of what is possible with coalescent and forward-time simulations. While coalescent simulations are way faster than forward-time, forward simulations let us add a lot more complexity to the simulation by relaxing the assumptions of the coalescent. 
        
It is important to note that coalescent simulators like `msprime` are being continually developed, and their capabilities expand all time. For example, `msprime` can simulate models with recombination and demography (relaxing the no-migration assumption). 

Additionally, hybrid simulations approaches are emerging that allow the selected parts of a simulation to be simulated forward-in-time, with the neutral parts added afterwards using coalescent. This is one of the purposes that `shadie` was built for, and it may greatly extend what we are able to model using simulations. 
</p>
</div>

## Appendix

### Burn-in

In [185]:
burn_time = NE*4
with shadie.Model() as burn:
    burn.initialize(
        chromosome=neutral_chrom, 
        sim_time=burn_time,
        mutation_rate=MUT, 
        recomb_rate=0,
        file_out="../data/wf_burn.trees",
    )

    burn.reproduction.wright_fisher(NE)
burn.run(np.random.randint(2**31))

In [134]:
mburn_time = Ne*4
with shadie.Model() as mburn:
    mburn.initialize(
        chromosome=neutral_chrom, 
        sim_time=mburn_time,
        mutation_rate=mut, 
        recomb_rate=0,
        file_out="../data/wf_mburn.trees",
    )

    mburn.reproduction.wright_fisher(Ne)
mburn.run(np.random.randint(2**31))